In [1]:
import os
from triangulator import create_triangulator_from_files

import sys
sys.path.append('../')
from utils.file_utils import create_capture_from_json

calibration_folder = '../calibration/calibration_data/'
intr_L = os.path.join(calibration_folder, 'calibration_L_frames=623_error=1.5265531081871597.pkl')
intr_R = os.path.join(calibration_folder, 'calibration_R_frames=624_error=0.99089142253755.pkl')
orientation_L = '../calibration/orientation_video_L.mp4.pkl'
orientation_R = '../calibration/orientation_video_R.mp4.pkl'

triangulator = create_triangulator_from_files(
    intr_files=[intr_L, intr_R],
    orientation_files=[orientation_L, orientation_R]
)

Image size is (1920, 1080)
Image size is (1920, 1080)
[array([[ 1.30658287],
       [-0.24183669],
       [-0.36029694]]), array([[0.8370347 ],
       [0.50877275],
       [1.42079534]])]


In [2]:
zero_point_L = (834, 751)
zero_point_R = (767, 887)

position_q = (triangulator.projection_matrices[0] @ [0, 0, 0, 1])
position_q = position_q / position_q[-1]
print(position_q)
print(triangulator.triangulate([zero_point_L, zero_point_R]))

[833.46269592 753.58957361   1.        ]
[-0.00070214  0.1955762   0.00064989]


In [ ]:
p1, p2, p3 = tuple(triangulator.projection_matrices[0])
print(p1)
print(p2)
print(p3)
print(triangulator.projection_matrices[0])

In [ ]:
caps = [
    create_capture_from_json(4, '../config/capture_params.json'),
    create_capture_from_json(2, '../config/capture_params.json')
]

In [ ]:
import cv2


while True:
    idx = 0
    for cap in caps:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = cv2.aruco.detectMarkers(
            image=gray,
            dictionary=cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
        )
        if ids is not None and 0 in ids:
            for corner_idx in range(len(ids)):
                if ids[corner_idx] == 0:
                    cv2.aruco.drawDetectedMarkers(
                        frame, 
                        corners=corners, 
                        ids=ids
                        )
                    break
        idx += 1
        cv2.imshow(f'cam{idx}', cv2.resize(src=frame, dsize=None, fx=0.35, fy=0.35))
    if cv2.waitKey(10) == 27:
        break


cv2.imshow('huh', frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

In [48]:
for cap in caps:
    cap.release()

cv2.destroyAllWindows()